# CMIP5 sandbox

>12 April 2021<br>
>Thomas Moore

In [1]:
import xarray as xr
from dask_jobqueue import PBSCluster
from dask.distributed import Client
import numpy as np
import xrft
import scipy

# Spin up a little cluster with `dask-jobqueue`

### Specs for an individual worker

In [2]:
walltime = '04:00:00'
cores = 2
memory = '8GB'

cluster = PBSCluster(walltime=str(walltime), cores=cores, memory=str(memory),
                     job_extra=['-l ncpus='+str(cores),'-l mem='+str(memory),
                                '-l storage=gdata/v14+scratch/v14+gdata/lp01+scratch/lp01+scratch/ux06+gdata/rr3'],
                     header_skip=["select"])

### Scale up to n workers

In [3]:
cluster.scale(jobs=4)

In [4]:
client = Client(cluster)
client

Client Scheduler: tcp://10.6.76.36:36919 Dashboard: http://10.6.76.36:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [5]:
client

Client Scheduler: tcp://10.6.76.36:36919 Dashboard: http://10.6.76.36:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


# Intake testbed

In [2]:
import intake

In [3]:
NCI_catalog = intake.open_catalog('/g/data/hh5/public/apps/nci-intake-catalogue/catalogue.yaml')

In [4]:
list(NCI_catalog)

['era5', 'era5_land', 'ecmwf', 'esgf']

In [11]:
list(NCI_catalog.esgf)

['cmip5',
 'cmip5_all',
 'cmip6',
 'cmip6_all',
 'cordex',
 'cordex_all',
 'cmip5_gr1p5',
 'cmip6_gr1p5']

In [35]:
NCI_catalog.esgf.cmip5_all.search(institute="MOHC",variable='psl',time_frequency='mon')

,unique
path,984
project,1
product,1
institute,1
model,4
experiment,65
time_frequency,1
realm,1
cmor_table,1
ensemble,32


In [38]:
NCI_catalog.esgf.cmip5_all.search(institute="MOHC",variable='psl',time_frequency='mon',experiment='rcp85').unique('path')

,unique
path,45
project,1
product,1
institute,1
model,2
experiment,1
time_frequency,1
realm,1
cmor_table,1
ensemble,4


In [41]:
NCI_catalog.esgf.cmip5_all.search(institute="MOHC",variable='psl',time_frequency='mon',experiment='rcp85',model='HadGEM2-CC',version='v20120531').unique('path')

{'path': {'count': 5,
  'values': ['/g/data/al33/replicas/CMIP5/output1/MOHC/HadGEM2-CC/rcp85/mon/atmos/Amon/r1i1p1/v20120531/psl/psl_Amon_HadGEM2-CC_rcp85_r1i1p1_200512-203011.nc',
   '/g/data/al33/replicas/CMIP5/output1/MOHC/HadGEM2-CC/rcp85/mon/atmos/Amon/r1i1p1/v20120531/psl/psl_Amon_HadGEM2-CC_rcp85_r1i1p1_203012-205511.nc',
   '/g/data/al33/replicas/CMIP5/output1/MOHC/HadGEM2-CC/rcp85/mon/atmos/Amon/r1i1p1/v20120531/psl/psl_Amon_HadGEM2-CC_rcp85_r1i1p1_205512-208011.nc',
   '/g/data/al33/replicas/CMIP5/output1/MOHC/HadGEM2-CC/rcp85/mon/atmos/Amon/r1i1p1/v20120531/psl/psl_Amon_HadGEM2-CC_rcp85_r1i1p1_208012-209912.nc',
   '/g/data/al33/replicas/CMIP5/output1/MOHC/HadGEM2-CC/rcp85/mon/atmos/Amon/r1i1p1/v20120531/psl/psl_Amon_HadGEM2-CC_rcp85_r1i1p1_210001-210012.nc']}}

# Build CCIA datasets

In [43]:
CCIA_HadGEM2_CC = NCI_catalog.esgf.cmip5_all.search(institute="MOHC",variable='psl',time_frequency='mon',experiment='rcp85',model='HadGEM2-CC',version='v20120531')

In [56]:
CCIA_ACCESS1_0 = NCI_catalog.esgf.cmip5_all.search(institute="CSIRO-BOM",variable='psl',time_frequency='mon',experiment='rcp85',model='ACCESS1.0',version='v20120727')

In [57]:
NCI_catalog.esgf.cmip5_all.search(institute="NSF-DOE-NCAR",variable='psl',time_frequency='mon',experiment='rcp85',model='CESM1(CAM5)').unique('path')

{'path': {'count': 5,
  'values': ['/g/data/al33/replicas/CMIP5/output1/NSF-DOE-NCAR/CESM1-CAM5/rcp85/mon/atmos/Amon/r1i1p1/v20130313/psl/psl_Amon_CESM1-CAM5_rcp85_r1i1p1_200601-210012.nc',
   '/g/data/al33/replicas/CMIP5/output1/NSF-DOE-NCAR/CESM1-CAM5/rcp85/mon/atmos/Amon/r2i1p1/v20130313/psl/psl_Amon_CESM1-CAM5_rcp85_r2i1p1_200601-210012.nc',
   '/g/data/al33/replicas/CMIP5/output1/NSF-DOE-NCAR/CESM1-CAM5/rcp85/mon/atmos/Amon/r3i1p1/v20130313/psl/psl_Amon_CESM1-CAM5_rcp85_r3i1p1_200601-210012.nc',
   '/g/data/al33/replicas/CMIP5/output1/NSF-DOE-NCAR/CESM1-CAM5/rcp85/mon/atmos/Amon/r3i1p1/v20140129/psl/psl_Amon_CESM1-CAM5_rcp85_r3i1p1_200601-210012.nc',
   '/g/data/al33/replicas/CMIP5/output1/NSF-DOE-NCAR/CESM1-CAM5/rcp85/mon/atmos/Amon/r3i1p1/v20140310/psl/psl_Amon_CESM1-CAM5_rcp85_r3i1p1_200601-210012.nc']}}

In [60]:
CCIA_CESM1_CAM5_r1i1p1 = NCI_catalog.esgf.cmip5_all.search(institute="NSF-DOE-NCAR",variable='psl',time_frequency='mon',experiment='rcp85',model='CESM1(CAM5)',ensemble='r1i1p1',version='v20130313')

In [61]:
CCIA_CESM1_CAM5_r2i1p1 = NCI_catalog.esgf.cmip5_all.search(institute="NSF-DOE-NCAR",variable='psl',time_frequency='mon',experiment='rcp85',model='CESM1(CAM5)',ensemble='r2i1p1',version='v20130313')

In [62]:
CCIA_CESM1_CAM5_r3i1p1 = NCI_catalog.esgf.cmip5_all.search(institute="NSF-DOE-NCAR",variable='psl',time_frequency='mon',experiment='rcp85',model='CESM1(CAM5)',ensemble='r2i1p1',version='v20140310')

In [69]:
NCI_catalog.esgf.cmip5_all.search(institute="CNRM-CERFACS",variable='psl',time_frequency='mon',experiment='rcp85',model='CNRM-CM5',ensemble='r10i1p1').unique('path')

{'path': {'count': 2,
  'values': ['/g/data/al33/replicas/CMIP5/output1/CNRM-CERFACS/CNRM-CM5/rcp85/mon/atmos/Amon/r10i1p1/v20110915/psl/psl_Amon_CNRM-CM5_rcp85_r10i1p1_200601-205512.nc',
   '/g/data/al33/replicas/CMIP5/output1/CNRM-CERFACS/CNRM-CM5/rcp85/mon/atmos/Amon/r10i1p1/v20110915/psl/psl_Amon_CNRM-CM5_rcp85_r10i1p1_205601-210012.nc']}}

In [63]:
NCI_catalog.esgf.cmip5_all.search(variable='psl',time_frequency='mon',experiment='rcp85').unique('institute')

{'institute': {'count': 23,
  'values': ['BCC',
   'BNU',
   'CCCMA',
   'CMCC',
   'CNRM-CERFACS',
   'CSIRO-BOM',
   'CSIRO-QCCCE',
   'FIO',
   'ICHEC',
   'INM',
   'IPSL',
   'LASG-CESS',
   'LASG-IAP',
   'MIROC',
   'MOHC',
   'MPI-M',
   'MRI',
   'NASA-GISS',
   'NCAR',
   'NCC',
   'NIMR-KMA',
   'NOAA-GFDL',
   'NSF-DOE-NCAR']}}

In [64]:
NCI_catalog.esgf.cmip5_all.search(variable='psl',time_frequency='mon',experiment='rcp85').unique('model')

{'model': {'count': 42,
  'values': ['ACCESS1.0',
   'ACCESS1.3',
   'BCC-CSM1.1',
   'BCC-CSM1.1(m)',
   'BNU-ESM',
   'CCSM4',
   'CESM1(BGC)',
   'CESM1(CAM5)',
   'CESM1(WACCM)',
   'CMCC-CESM',
   'CMCC-CM',
   'CMCC-CMS',
   'CNRM-CM5',
   'CSIRO-Mk3L-1-2',
   'CanESM2',
   'EC-EARTH',
   'FGOALS-g2',
   'FGOALS-s2',
   'FIO-ESM',
   'GFDL-CM3',
   'GFDL-ESM2G',
   'GFDL-ESM2M',
   'GISS-E2-H',
   'GISS-E2-H-CC',
   'GISS-E2-R',
   'GISS-E2-R-CC',
   'HadGEM2-AO',
   'HadGEM2-CC',
   'HadGEM2-ES',
   'INM-CM4',
   'IPSL-CM5A-LR',
   'IPSL-CM5A-MR',
   'IPSL-CM5B-LR',
   'MIROC-ESM',
   'MIROC-ESM-CHEM',
   'MIROC5',
   'MPI-ESM-LR',
   'MPI-ESM-MR',
   'MRI-CGCM3',
   'MRI-ESM1',
   'NorESM1-M',
   'NorESM1-ME']}}

In [9]:
NCI_catalog.esgf.search(variable='psl').unique('path')

TypeError: search() got an unexpected keyword argument 'variable'

In [30]:
result = cordex.search(variable='psl')
result

,unique
project,1
product,1
domain,2
institute,3
driving_model,12
experiment,4
ensemble,1
rcm_name,7
rcm_version,2
time_frequency,5


In [39]:
cordex.search(institute='UNSW', time_frequency='day', variable='psl', domain='AUS-44i', require_all_on=['rcm_name']).unique('path')

{'path': {'count': 66,
  'values': ['/g/data/rr3/publications/CORDEX/output/AUS-44i/UNSW/CSIRO-BOM-ACCESS1-0/historical/r1i1p1/UNSW-WRF360J/v1/day/psl/v20180614/psl_AUS-44i_CSIRO-BOM-ACCESS1-0_historical_r1i1p1_UNSW-WRF360J_v1_day_19510101-19551231.nc',
   '/g/data/rr3/publications/CORDEX/output/AUS-44i/UNSW/CSIRO-BOM-ACCESS1-0/historical/r1i1p1/UNSW-WRF360J/v1/day/psl/v20180614/psl_AUS-44i_CSIRO-BOM-ACCESS1-0_historical_r1i1p1_UNSW-WRF360J_v1_day_19560101-19601231.nc',
   '/g/data/rr3/publications/CORDEX/output/AUS-44i/UNSW/CSIRO-BOM-ACCESS1-0/historical/r1i1p1/UNSW-WRF360J/v1/day/psl/v20180614/psl_AUS-44i_CSIRO-BOM-ACCESS1-0_historical_r1i1p1_UNSW-WRF360J_v1_day_19610101-19651231.nc',
   '/g/data/rr3/publications/CORDEX/output/AUS-44i/UNSW/CSIRO-BOM-ACCESS1-0/historical/r1i1p1/UNSW-WRF360J/v1/day/psl/v20180614/psl_AUS-44i_CSIRO-BOM-ACCESS1-0_historical_r1i1p1_UNSW-WRF360J_v1_day_19660101-19701231.nc',
   '/g/data/rr3/publications/CORDEX/output/AUS-44i/UNSW/CSIRO-BOM-ACCESS1-0/histori

# $The\ End$

In [16]:
cluster.close()
client.close()